In [1]:
from datetime import datetime, timedelta
import os
import re
import glob
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import wget
import numpy as np

In [2]:
conf_df = pd.read_csv(r'D:\Python coding\Datasets and codes\23-05-2021\COVID-19-master\COVID-19-master\csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv(r'D:\Python coding\Datasets and codes\23-05-2021\COVID-19-master\COVID-19-master\csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv(r'D:\Python coding\Datasets and codes\23-05-2021\COVID-19-master\COVID-19-master\csse_covid_19_data\csse_covid_19_time_series\time_series_covid19_recovered_global.csv')

In [3]:
conf_df.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,63045,63355,63412,63484,63598,63819,64122,64575,65080,65486
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,131890,131939,131978,132015,132032,132071,132095,132118,132153,132176
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,124889,125059,125194,125311,125485,125693,125896,126156,126434,126651


In [4]:
deaths_df.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2721,2730,2733,2742,2745,2751,2762,2772,2782,2792
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2426,2427,2429,2432,2435,2436,2438,2440,2441,2442
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3355,3360,3366,3374,3381,3388,3395,3401,3405,3411


In [5]:
recovered_df.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,54619,54634,54663,54686,55010,55118,55529,55687,55790,55889
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,121122,122105,123081,124312,125419,126405,127240,127869,128425,128601
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,87003,87137,87251,87359,87476,87609,87746,87902,88066,88208


In [6]:
conf_df.columns[:4]

Index(['Province/State', 'Country/Region', 'Lat', 'Long'], dtype='object')

In [7]:
deaths_df.columns[:4]

Index(['Province/State', 'Country/Region', 'Lat', 'Long'], dtype='object')

In [8]:
recovered_df.columns[:4]

Index(['Province/State', 'Country/Region', 'Lat', 'Long'], dtype='object')

In [9]:
dates = deaths_df.columns[4:]
dates

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '5/13/21', '5/14/21', '5/15/21', '5/16/21', '5/17/21', '5/18/21',
       '5/19/21', '5/20/21', '5/21/21', '5/22/21'],
      dtype='object', length=487)

In [10]:
conf_df_melted = conf_df.melt(id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'],
                             value_vars = dates,
                             var_name = 'Date',
                             value_name = 'Confirmed')
conf_df_melted.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [11]:
deaths_df_melted = deaths_df.melt(id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'],
                                  value_vars = dates,
                                  var_name = 'Date',
                                  value_name = 'Deaths')
deaths_df_melted.head()

,Province/State,Country/Region,Lat,Long,Date,Deaths
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [12]:
recovered_df_melted = recovered_df.melt(id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'],
                                 value_vars = dates,
                                 var_name = 'Date',
                                 value_name = 'Recovered')
recovered_df_melted.head()

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [13]:
print(conf_df_melted.shape)
print(deaths_df_melted.shape)
print(recovered_df_melted.shape)

(133925, 6)
(133925, 6)
(126620, 6)


In [14]:
merged_df = pd.merge(left=conf_df_melted, right=deaths_df_melted, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

merged_df = pd.merge(left=merged_df, right=recovered_df_melted, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Recovered'] = merged_df['Recovered'].fillna(0).astype('int')

merged_df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0


In [15]:
#Fixing Country Names

merged_df['Country/Region'] = merged_df['Country/Region'].replace('Korea, South', 'South Korea')
merged_df.loc[merged_df['Province/State']=='Greenland', 'Country/Region'] = 'Greenland'
merged_df['Country/Region'] = merged_df['Country/Region'].replace('Mainland China', 'China')

In [16]:
#merged_df.to_csv("merged_df.csv")

In [17]:
# removing canada's recovered values
merged_df = merged_df[merged_df['Province/State'].str.contains('Recovered')!=True]
merged_df

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0
...,...,...,...,...,...,...,...,...
133920,NaN,Vietnam,14.058324,108.277199,2021-05-22,5119,42,2721
133921,NaN,West Bank and Gaza,31.952200,35.233200,2021-05-22,305201,3459,297201
133922,NaN,Yemen,15.552727,48.516388,2021-05-22,6649,1304,3201
133923,NaN,Zambia,-13.133897,27.849332,2021-05-22,93106,1267,91102


In [18]:
# removing county wise data to avoid double counting
merged_df = merged_df[merged_df['Province/State'].str.contains(',')!=True]
merged_df

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0
...,...,...,...,...,...,...,...,...
133920,NaN,Vietnam,14.058324,108.277199,2021-05-22,5119,42,2721
133921,NaN,West Bank and Gaza,31.952200,35.233200,2021-05-22,305201,3459,297201
133922,NaN,Yemen,15.552727,48.516388,2021-05-22,6649,1304,3201
133923,NaN,Zambia,-13.133897,27.849332,2021-05-22,93106,1267,91102


In [19]:
merged_df['Active'] = merged_df['Confirmed'] - merged_df['Deaths'] - merged_df['Recovered']
merged_df

<ipython-input-19-828993d1f338>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Active'] = merged_df['Confirmed'] - merged_df['Deaths'] - merged_df['Recovered']


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0
...,...,...,...,...,...,...,...,...,...
133920,NaN,Vietnam,14.058324,108.277199,2021-05-22,5119,42,2721,2356
133921,NaN,West Bank and Gaza,31.952200,35.233200,2021-05-22,305201,3459,297201,4541
133922,NaN,Yemen,15.552727,48.516388,2021-05-22,6649,1304,3201,2144
133923,NaN,Zambia,-13.133897,27.849332,2021-05-22,93106,1267,91102,737


In [20]:
merged_df[['Province/State']] = merged_df[['Province/State']].fillna('')
merged_df

C:\Users\rebek\anaconda3\envs\tensor\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0
1,,Albania,41.153300,20.168300,2020-01-22,0,0,0,0
2,,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0
3,,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0
4,,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0
...,...,...,...,...,...,...,...,...,...
133920,,Vietnam,14.058324,108.277199,2021-05-22,5119,42,2721,2356
133921,,West Bank and Gaza,31.952200,35.233200,2021-05-22,305201,3459,297201,4541
133922,,Yemen,15.552727,48.516388,2021-05-22,6649,1304,3201,2144
133923,,Zambia,-13.133897,27.849332,2021-05-22,93106,1267,91102,737


In [21]:
cols = ['Confirmed', 'Deaths', 'Recovered', 'Active']

# replacing all Nan with 0s
merged_df[cols] = merged_df[cols].fillna(0)
merged_df


C:\Users\rebek\anaconda3\envs\tensor\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0
1,,Albania,41.153300,20.168300,2020-01-22,0,0,0,0
2,,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0
3,,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0
4,,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0
...,...,...,...,...,...,...,...,...,...
133920,,Vietnam,14.058324,108.277199,2021-05-22,5119,42,2721,2356
133921,,West Bank and Gaza,31.952200,35.233200,2021-05-22,305201,3459,297201,4541
133922,,Yemen,15.552727,48.516388,2021-05-22,6649,1304,3201,2144
133923,,Zambia,-13.133897,27.849332,2021-05-22,93106,1267,91102,737


In [23]:

merged_df['Recovered'] = merged_df['Recovered'].astype(int)

print(merged_df.shape)
merged_df

(132951, 9)


<ipython-input-23-f71c0ef3da18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Recovered'] = merged_df['Recovered'].astype(int)


,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
0,,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0
1,,Albania,41.153300,20.168300,2020-01-22,0,0,0,0
2,,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0
3,,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0
4,,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0
...,...,...,...,...,...,...,...,...,...
133920,,Vietnam,14.058324,108.277199,2021-05-22,5119,42,2721,2356
133921,,West Bank and Gaza,31.952200,35.233200,2021-05-22,305201,3459,297201,4541
133922,,Yemen,15.552727,48.516388,2021-05-22,6649,1304,3201,2144
133923,,Zambia,-13.133897,27.849332,2021-05-22,93106,1267,91102,737


In [24]:
merged_df.to_csv('Merged_data.csv', index=False)